# Swedish medical dataset

In [2]:
from datasets import load_dataset

wnut = load_dataset("wnut_17")
wnut['train'][0]
#    dataset w  bad formatting
ner_dataset = load_dataset('swedish_medical_ner', 'wiki')
ner_dataset['train'][0]



/Users/jinyan/opt/anaconda3/envs/saman_med_ner/lib/python3.10/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
Generating train split: 100%|██████████| 48720/48720 [00:00<00:00, 486480.19 examples/s]


In [4]:
wnut['train'][0]

{'id': '0',
 'tokens': ['@paulwalk',
  'It',
  "'s",
  'the',
  'view',
  'from',
  'where',
  'I',
  "'m",
  'living',
  'for',
  'two',
  'weeks',
  '.',
  'Empire',
  'State',
  'Building',
  '=',
  'ESB',
  '.',
  'Pretty',
  'bad',
  'storm',
  'here',
  'last',
  'evening',
  '.'],
 'ner_tags': [0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  7,
  8,
  8,
  0,
  7,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0]}

In [19]:
ner_dataset['train'][:10]

{'sid': ['wiki_0',
  'wiki_1',
  'wiki_2',
  'wiki_3',
  'wiki_4',
  'wiki_5',
  'wiki_6',
  'wiki_7',
  'wiki_8',
  'wiki_9'],
 'sentence': ['{kropp} beskrivs i till exempel människokroppen, anatomi och f',
  'sju miljoner år gammalt hominint {kranium}, klassificerad som ',
  ' autosomer och ett par könskromosomer. Varje {kromosom} består',
  ' {kromosom} består av en DNA-molekyl och {protein}. En DNA-molek',
  'tikel:Människans {skelett} Människans skelett\xa0är det\xa0skelett\xa0s',
  'os\xa0människor. En vuxen människas {skelett} består av 206 till ',
  '{lett} består av 206 till 220 {ben}, beroende på hur man räknar.',
  'v kroppsvikten.Ett nyfött barn har ca 300 {ben} i kroppen vilk',
  'kollektivet i mindre bitar såsom länder > städer > orter {Hud}',
  'sdjur. {Huden} utgör ett mekaniskt skydd mot omvärlden och bid'],
 'entities': [{'start': [0], 'end': [7], 'text': ['kropp'], 'type': [2]},
  {'start': [33], 'end': [42], 'text': ['kranium'], 'type': [2]},
  {'start': [45], 'end'

In [15]:
all_entities = set()
for row in ner_dataset['train'][:100]:
    print(row)
    for entity in row['entities']['text']:
        print(entity)
        all_entities.add(entity)

sid


TypeError: string indices must be integers

In [10]:
# count how many entities have more than one word
entities_with_more_than_one_word = []
for entity in all_entities:
    if len(entity.split()) > 1:
        entities_with_more_than_one_word.append(entity)

In [11]:
entities_with_more_than_one_word[:10]

['kant}. Mellan {dura ',
 'akut stressyndrom',
 'som halotan',
 'ovopulmon Novolizer',
 'glatta muskulaturen} i {aorta',
 'sympatiska nervsystemet}, beroende på om kroppen eller {psy',
 'skt stressyndrom',
 'n STADA',
 'aponeuros} täcker {Musculus rectus abdominis',
 'kromosom} består av en DNA-molekyl och {protein']

## Transform dataset to BIO format for huggingface

In [30]:
import spacy

label2id = {
    "O": 0,  # Outside of a named entity
    "B-DISORDER": 1,
    "I-DISORDER": 2,
    "B-DRUG": 3,
    "I-DRUG": 4,
    "B-BODY": 5,
    "I-BODY": 6,
}
id2label = {
    0: "O",
    1: "B-DISORDER",
    2: "I-DISORDER",
    3: "B-DRUG",
    4: "I-DRUG",
    5: "B-BODY",
    6: "I-BODY",
    
}
def clean_space_characters(text):
    return text.replace(u'\xa0', u' ')

def transform_dataset_row(sentence, nlp):
    sentence = clean_space_characters(sentence)
    doc = nlp(sentence)

    tokens = []
    ner_tags = []
    current_entity_type = None
    start_of_entity = False

    for token in doc:
        if token.text == '(':
            current_entity_type = ("B-DISORDER", "I-DISORDER")
            start_of_entity = True
        elif token.text == '[':
            current_entity_type = ("B-DRUG", "I-DRUG")
            start_of_entity = True
        elif token.text == '{':
            current_entity_type = ("B-BODY", "I-BODY")
            start_of_entity = True
        elif token.text in [')', ']', '}'] and current_entity_type:
            current_entity_type = None
        else:
            if current_entity_type:
                if start_of_entity == True:
                    ner_tags.append(NER_TAGS[current_entity_type[0]])
                    start_of_entity = False
                else:
                    ner_tags.append(NER_TAGS[current_entity_type[1]])
            else:
                ner_tags.append(NER_TAGS["O"])
            tokens.append(token.text)

    return {'tokens': tokens, 'ner_tags': ner_tags}

# Example usage
nlp = spacy.blank("en")

sentence = 'tikel:Människans {skelett} Människans skelett\xa0är det\xa0skelett\xa0s'
output = transform_dataset_row(sentence,nlp)
print(output)


{'tokens': ['tikel', ':', 'Människans', 'skelett', 'Människans', 'skelett', 'är', 'det', 'skelett', 's'], 'ner_tags': [0, 0, 0, 5, 0, 0, 0, 0, 0, 0]}


##

In [32]:
# Transform ner_dataset to a new format
from datasets import Dataset
new_train_dataset = []
for row in ner_dataset['train']:
    new_train_dataset.append(transform_dataset_row(row['sentence'],nlp))
dataset = Dataset.from_list(new_train_dataset)

In [38]:
dataset[0]

{'tokens': ['kropp',
  'beskrivs',
  'i',
  'till',
  'exempel',
  'människokroppen',
  ',',
  'anatomi',
  'och',
  'f'],
 'ner_tags': [5, 0, 0, 0, 0, 0, 0, 0, 0, 0]}

In [46]:
dataset = dataset.train_test_split(test_size=0.1)

In [47]:
train_dataset = dataset['train']
test_dataset = dataset['test']
len(train_dataset), len(test_dataset)

(43848, 4872)

In [48]:
# Load model directly
from transformers import AutoTokenizer, AutoModelForTokenClassification, TrainingArguments, Trainer
model_id = 'KB/bert-base-swedish-cased'
tokenizer = AutoTokenizer.from_pretrained(model_id)
model = AutoModelForTokenClassification.from_pretrained(
    model_id, num_labels=13, id2label=id2label, label2id=label2id
)

None of PyTorch, TensorFlow >= 2.0, or Flax have been found. Models won't be available and only tokenizers, configuration and file/data utilities can be used.


ImportError: 
AutoModelForMaskedLM requires the PyTorch library but it was not found in your environment. Checkout the instructions on the
installation page: https://pytorch.org/get-started/locally/ and follow the ones that match your environment.
Please note that you may need to restart your runtime after installation.


In [ ]:

def tokenize_and_align_labels(examples, tokenizer, tokens_get_same_label_as_first_token=True):
    tokenized_inputs = tokenizer(examples["tokens"], truncation=True, is_split_into_words=True)

    labels = []
    for i, label in enumerate(examples["ner_tags"]):
        word_ids = tokenized_inputs.word_ids(batch_index=i)  # Map tokens to their respective word.
        previous_word_idx = None
        label_ids = []
        for word_idx in word_ids:  # Special tokens get -100.
            if word_idx is None:
                label_ids.append(-100)
            elif word_idx != previous_word_idx:  # First token of a word gets its label.
                label_ids.append(label[word_idx])
            else:
                # Subsequent tokens of a word get the same label as the first token of the word.
                if tokens_get_same_label_as_first_token:
                    label_ids.append(label[word_idx])
                else:
                    label_ids.append(-100)
            previous_word_idx = word_idx
        labels.append(label_ids)

    tokenized_inputs["labels"] = labels
    return tokenized_inputs


In [ ]:
import evaluate

seqeval = evaluate.load("seqeval")

In [ ]:
example = train_dataset[0]


In [ ]:
import numpy as np
label_list = label2id.keys()
labels = [label_list[i] for i in example[f"ner_tags"]]


def compute_metrics(p):
    predictions, labels = p
    predictions = np.argmax(predictions, axis=2)

    true_predictions = [
        [label_list[p] for (p, l) in zip(prediction, label) if l != -100]
        for prediction, label in zip(predictions, labels)
    ]
    true_labels = [
        [label_list[l] for (p, l) in zip(prediction, label) if l != -100]
        for prediction, label in zip(predictions, labels)
    ]

    results = seqeval.compute(predictions=true_predictions, references=true_labels)
    return {
        "precision": results["overall_precision"],
        "recall": results["overall_recall"],
        "f1": results["overall_f1"],
        "accuracy": results["overall_accuracy"],
    }

In [ ]:
from transformers import DataCollatorForTokenClassification

data_collator = DataCollatorForTokenClassification(tokenizer=tokenizer)

In [ ]:
training_args = TrainingArguments(
    output_dir="swe_medical_ner",
    learning_rate=2e-5,
    per_device_train_batch_size=16,
    per_device_eval_batch_size=16,
    num_train_epochs=2,
    weight_decay=0.01,
    evaluation_strategy="epoch",
    save_strategy="epoch",
    load_best_model_at_end=True,
    push_to_hub=True,
)

trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=train_dataset,
    eval_dataset=test_dataset,
    tokenizer=tokenizer,
    data_collator=data_collator,
    compute_metrics=compute_metrics,
)

trainer.train()